In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ELU
from keras.layers import Lambda, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import ELU
from keras.optimizers import Adam
%matplotlib inline
# np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [2]:
# Global Vars
img = (cv.imread('../train/photos/images/gray_Road_Only/0.jpg')).shape
img_height = img[0]
img_width = img[1]
img_channels = img[2]
print(img, img_height, img_width, img_channels)

(120, 300, 3) 120 300 3


In [3]:
def adjust_Value(prvs_FRAME, curr_FRAME, HSV_value):
    prvs_FRAME = cv.cvtColor(prvs_FRAME, cv.COLOR_BGR2HSV)
    curr_FRAME = cv.cvtColor(curr_FRAME, cv.COLOR_BGR2HSV)
    prvs_FRAME[...,2] = prvs_FRAME[...,2]*HSV_value
    curr_FRAME[...,2] = curr_FRAME[...,2]*HSV_value
    return prvs_FRAME, curr_FRAME

In [4]:
def opticalFlow_DENSE(prvs_FRAME, curr_FRAME, dynamic_sat):
    # flow_mat = None
    # image_scale = 0.5
    # pyr_images = 1 # was 3
    # win_size = 15
    # pyr_iterations = 2 # was 3
    # poly_expans = 5
    # std = 1.3 # was 1.2
    
    flow_mat = None
    image_scale = 0.5
    pyr_images = 1 # was 3
    win_size = 15
    pyr_iterations = 2 # was 3
    poly_expans = 5
    std = 1.3 # was 1.2

    hsv = np.zeros_like(prvs_FRAME)
    if dynamic_sat:
        hsv[..., 1] = prvs_FRAME[..., 1]
    else:
        hsv[...,1] = 255
    prvs_FRAME = cv.cvtColor(prvs_FRAME,cv.COLOR_BGR2GRAY)
    curr_FRAME = cv.cvtColor(curr_FRAME,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_FRAME,curr_FRAME,flow_mat,image_scale,pyr_images,win_size,pyr_iterations,poly_expans,std,0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    hsv = np.asarray(hsv, dtype=np.float32)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    return bgr

In [5]:
def getData(train_meta, kLoopCount, train_index, getSpeed, for_training):
    global img_height, img_width, img_channels
    opt_flows = np.empty((0, img_height, img_width, img_channels), dtype='float32')
    speed = np.empty((0))
    for k in range(kLoopCount):
        HSV_value = np.random.uniform(low=.2, high=1.2)
        #print(train_index)
        curr_FRAME = cv.imread(train_meta['image_path'][train_index])
        prvs_FRAME = cv.imread(train_meta['image_path'][train_index-1])
        if getSpeed:
            speed1 = train_meta['speed'][train_index]
            speed2 = train_meta['speed'][train_index-1]
            speed3 = np.mean((speed1, speed2))
            speed = np.append(speed, speed3)
        if for_training:
            prvs_FRAME, curr_FRAME = adjust_Value(prvs_FRAME, curr_FRAME, HSV_value)
        opt_flow = opticalFlow_DENSE(prvs_FRAME, curr_FRAME, False)
        #opt_flow.dtype = 'uint8'
        opt_flows = np.append(opt_flows, [opt_flow], axis=0)
    if getSpeed:
        return opt_flows, speed
    else:
        return opt_flows

In [6]:
def getTrainingData(train_meta, train_index, batchSize):
    global img_height, img_width, img_channels
    get_speed = True
    for_training = True
    while True:
        opt_flow_arr = np.empty((0, img_height, img_width, img_channels))
        speed_arr = np.empty((0))
        index2 = 0
        for i in range(batchSize):
            #print(index2)
            index = np.random.randint(0, train_index.shape[0])
            opt_flow, speed = getData(train_meta, 1, train_index[index], get_speed, for_training)
            opt_flow_arr = np.append(opt_flow_arr, opt_flow, axis=0)
            speed_arr = np.append(speed_arr, speed)
            index2 = index2 + 1
        #print(opt_flow_arr.shape)
        yield opt_flow_arr, speed_arr

In [7]:
def getValData(train_meta, val_index, batchSize, getSpeed):
    for_training = False
    while True:
        index = 0
        for i in range(len(val_index)):
            if getSpeed:
                opt_flow, speed = getData(train_meta, 1, val_index[index], getSpeed, for_training)
            else:
                opt_flow = getData(train_meta, 1, val_index[index], getSpeed, for_training)
            #opt_flow_arr = np.append(opt_flow_arr, opt_flow)
            index = index + 1
            if getSpeed:
                yield opt_flow, speed
            else:
                yield opt_flow

In [8]:
def createCNNModel():
    global img_height, img_width, img_channels
#     model = Sequential()
#     model.add(Lambda(lambda x: x / 127.5 - 1, input_shape=(img_height, img_width, img_channels)))
#     model.add(Conv2D(filters=40, kernel_size=(5,5), strides=(3,3), activation='elu'))
#     #model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Conv2D(filters=20, kernel_size=(5,5), strides=(2,2), activation='elu'))
#     #model.add(MaxPooling2D(pool_size=(2,2)))
#     #model.add(Dropout(rate=0.25))
#     model.add(Flatten())
#     model.add(Dense(units=128, activation='elu'))
#     #model.add(Dropout(rate=0.3))
#     model.add(Dense(units=64, activation='elu'))
#     model.add(Dense(units=15, activation='elu'))
#     model.add(Dense(units=1, activation='elu'))
#     opt = Adam(learning_rate=1e-4, epsilon=1e-08)
#     model.compile(optimizer=opt, loss='mse')
    
    model = Sequential()
    # normalization    
    # perform custom normalization before lambda layer in network
    model.add(Lambda(lambda x: x/ 127.5 - 1, input_shape = (img_height, img_width, img_channels)))

    model.add(Conv2D(24, (5, 5), 
                     strides=(2,2), 
                     padding = 'valid',
                     kernel_initializer = 'he_normal'))
    
    
    model.add(ELU())    
    model.add(Conv2D(36, (5, 5), 
                     strides=(2,2), 
                     padding = 'valid',
                     kernel_initializer = 'he_normal'))
    
    model.add(ELU())    
    model.add(Conv2D(48, (5, 5), 
                     strides=(2,2), 
                     padding = 'valid',
                     kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 3), 
                     strides = (1,1), 
                     padding = 'valid',
                     kernel_initializer = 'he_normal'))
    
    model.add(ELU())              
    model.add(Conv2D(64, (3, 3), 
                     strides= (1,1), 
                     padding = 'valid',
                     kernel_initializer = 'he_normal'))
              
              
    model.add(Flatten())
    model.add(ELU())
    model.add(Dense(100, kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dense(50, kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dense(10, kernel_initializer = 'he_normal'))
    model.add(ELU())
    
    # do not put activation at the end because we want to exact output, not a class identifier
    model.add(Dense(1, name = 'output', kernel_initializer = 'he_normal'))
    
    # LR was 1e-4 and epsilon was 1e-08
    adam = Adam(learning_rate=1e-3, epsilon=1e-07)
    model.compile(optimizer = adam, loss = 'mse')
    
    return model

In [9]:
def a_print_fn(x):
    global train_name
    dic = {'lambd':'L', 'conv2':'C', 'flatt':'F', 'dense':'De', 'dropo':'D'}
    dic2 = {'C':4, 'De':0}
    dic3 = {',':0, ')':1, ' ':0}
    if x[:5] in dic:
        symbol = dic[x[:5]]
        symbol_unit = ''
        index = 13
        commaSpace_count = 0
        if symbol in dic2:
            while True:
                symbol_num = x[index]
                if symbol_num not in dic3:
                    if commaSpace_count == dic2[symbol]:
                        symbol_unit = symbol_unit + symbol_num
                    index = index + 1
                else:
                    if commaSpace_count == dic2[symbol]:
                        break
                    commaSpace_count = commaSpace_count+1
                    index = index + 1
        train_name = train_name + '-' + symbol + symbol_unit
        print(train_name)
#     print(x)

In [10]:
model_name = 'gray_Road_Only'
batch_size = 20
epoch_num = 20
steps_per_epoch = 400

train_meta_loc = '../train/text/CSV'
train_meta_loc = os.path.join(train_meta_loc, 'trainGrayRoadOnly_meta.csv')
test_meta_loc = '../test/text/CSV'
test_meta_loc = os.path.join(test_meta_loc, 'testGrayRoadOnly_meta.csv')

asset_path = '../train/assets'
#train_name = f'model={model_name}_C24_C36_C48_D50_C64_C64_De100_De50_De10_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}'
#train_name = f'model={model_name}2_elu_C40P_C20P_D25_De128_D30_De64_De15_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}'
#train_name = f'model={model_name}_elu_C40_C20_D25_De128_D30_De64_De15_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}__3'
#train_name = f'model={model_name}_elu_C40_C20_D35_De128_De64_De15_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}__4'
#train_name = f'model={model_name}_elu_C40_C20_De128_De64_De15_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}__2'
train_name = f'model={model_name}-elu'
model = createCNNModel()
model.summary(line_length=100, positions=[0.06,0.55,0.67,0.0], print_fn=a_print_fn)
model = 0
train_name_end = f'-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}__2'
train_name = train_name + train_name_end
train_path = os.path.join(asset_path, train_name)

model_train_meta_loc = os.path.join(train_path, 'train_meta.csv')
model_test_meta_loc = os.path.join(train_path, 'test_meta.csv')
test_result_path = os.path.join(train_path, 'test.txt')
history_loc = os.path.join(train_path, 'history.p')
weights_loc = os.path.join(train_path, 'weights.h5')
tensorboard_loc = os.path.join(train_path, train_name)
print(history_loc, '\n', weights_loc, '\n', tensorboard_loc)

model=gray_Road_Only-elu-L
model=gray_Road_Only-elu-L-C24
model=gray_Road_Only-elu-L-C24-C36
model=gray_Road_Only-elu-L-C24-C36-C48
model=gray_Road_Only-elu-L-C24-C36-C48-D
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50
model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10
../train/assets/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch_size=20-num_epoch=20-steps_per_epoch=400__2/history.p 
 ../train/assets/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch_size=20-num_epoch=20-steps_per_epoch=400__2/weights.h5 
 ../train/assets/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch_size=20-num_epoch=20-steps_per_epoch=400__2/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch

In [11]:
def start():
    global train_meta_loc, batch_size, epoch_num, steps_per_epoch, train_path
    train_meta = pd.read_csv(train_meta_loc)
    
    data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    dataset_frames_num = train_meta.shape[0]
    train_frames_num = int(.78*(dataset_frames_num-1))
    val_frames_num = int((dataset_frames_num - train_frames_num))
    
    
    train_index = data_index[:train_frames_num]
    val_index = data_index[train_frames_num:(train_frames_num+val_frames_num)]
    print(train_index.shape, val_index.shape)
    
    if not os.path.exists(train_path):
        os.makedirs(train_path)
    
    valid_generator = getValData(train_meta, val_index, batch_size, True)
    val_size = len(val_index)
    earlyStopping = EarlyStopping(monitor='val_loss', 
                                  patience=1, 
                                  verbose=1, 
                                  min_delta = 0.23,
                                  mode='min',)
    
    modelCheckpoint = ModelCheckpoint(weights_loc, 
                                      monitor = 'val_loss', 
                                      save_best_only = True, 
                                      mode = 'min', 
                                      verbose = 1,
                                     save_weights_only = True)
    
    tensorboard = TensorBoard(log_dir=tensorboard_loc, histogram_freq=0,
                                write_graph=True, write_images=True)
    #callbacks_list = [modelCheckpoint, tensorboard, earlyStopping]
    callbacks_list = [modelCheckpoint, tensorboard]
    
    model = createCNNModel()
    
    train_size = len(train_index)
    train_generator = getTrainingData(train_meta, train_index, batch_size)
    history = model.fit_generator(
            train_generator, 
            steps_per_epoch = steps_per_epoch, 
            epochs = epoch_num,
            callbacks = callbacks_list,
            verbose = 1,
            validation_data = valid_generator,
            validation_steps = val_size)

    print(history)
    pickle.dump(history.history, open(history_loc, "wb"))
    
    return model, history.history, valid_generator, val_size

In [12]:
model, history, valid_generator, val_size = start()

(15911,) (4488,)
Epoch 1/20
400/400 [==============================] - 142s 356ms/step - loss: 72.9582 - val_loss: 296.2667

Epoch 00001: val_loss improved from inf to 296.26672, saving model to ../train/assets/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch_size=20-num_epoch=20-steps_per_epoch=400__2/weights.h5
Epoch 2/20
400/400 [==============================] - 138s 345ms/step - loss: 30.1403 - val_loss: 216.1462

Epoch 00002: val_loss improved from 296.26672 to 216.14621, saving model to ../train/assets/model=gray_Road_Only-elu-L-C24-C36-C48-D-C64-C64-F-De100-De50-De10-batch_size=20-num_epoch=20-steps_per_epoch=400__2/weights.h5
Epoch 3/20
400/400 [==============================] - 138s 346ms/step - loss: 25.7041 - val_loss: 267.1067

Epoch 00003: val_loss did not improve from 216.14621
Epoch 4/20
400/400 [==============================] - 138s 345ms/step - loss: 23.5478 - val_loss: 165.6489

Epoch 00004: val_loss improved from 216.14621 to 165.64890, savi

In [ ]:
#history = pickle.load(open(history_loc, "rb" ))
model = createCNNModel()
model.load_weights(weights_loc)
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer = adam, loss = 'mse')

In [13]:
train_meta = pd.read_csv(train_meta_loc)

data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
dataset_frames_num = train_meta.shape[0]
#train_frames_num = int(.8*(dataset_frames_num-1))
train_frames_num = int(.6*(dataset_frames_num-1))
val_frames_num = int(.5*((dataset_frames_num-1) - train_frames_num))
test_frames_num = int(((dataset_frames_num-1) - (train_frames_num + val_frames_num)))
print(train_frames_num, val_frames_num, test_frames_num)

train_index = data_index[:train_frames_num]
val_index = data_index[train_frames_num:(train_frames_num+val_frames_num)]
test_index = data_index[(train_frames_num+val_frames_num):]
train_generator = getValData(train_meta, train_index, batch_size, True)
valid_generator = getValData(train_meta, val_index, batch_size, True)
test_generator = getValData(train_meta, test_index, batch_size, True)
train_size = len(train_index)
val_size = len(val_index)
test_size = len(test_index)

12239 4080 4080


In [17]:
train_score = model.evaluate(train_generator, steps=train_size)
print('train score:', train_score)

12239/12239 [==============================] - 115s 9ms/step
train score: 4.114180088043213


In [18]:
val_score = model.evaluate(valid_generator, steps=val_size)
print('val score:', val_score)

4080/4080 [==============================] - 39s 10ms/step
val score: 27.780153274536133


In [19]:
test_score = model.evaluate(test_generator, steps=test_size)
print('test score:', test_score)

4080/4080 [==============================] - 38s 9ms/step
test score: 0.2292296290397644


In [ ]:
plt.plot(history['loss'], 'ro')
plt.plot(history['val_loss'], 'go')

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
print(fig_size)
fig_size[0] = 30
fig_size[1] = 30
plt.rcParams["figure.figsize"] = fig_size
fig_size2 = plt.rcParams["figure.figsize"]
print(fig_size2)

In [ ]:
train_meta = pd.read_csv(train_meta_loc)

data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
dataset_frames_num = train_meta.shape[0]
train_frames_num = int(.70*(dataset_frames_num-1))

train_index = data_index[:train_frames_num]
val_index = data_index[train_frames_num:]
print(train_index, val_index)
plt.plot(train_meta['speed'][train_index], 'ro')
plt.plot(train_meta['speed'][val_index], 'go')

In [ ]:
def predictSpeed(a_meta, a_meta_loc, getSpeed, setError):
    global model_train_meta_loc, model_test_meta_loc, batch_size, weights_loc

    dataset_frames_num = a_meta.shape[0]
    print(dataset_frames_num)
    
    data_index = np.arange(1, (dataset_frames_num))
    data_length = data_index.shape[0]
    
    data_gen = getValData(a_meta, data_index, batch_size, getSpeed)
    
    model = createCNNModel()
    model.load_weights(weights_loc)
    predicted_speed = model.predict(data_gen, steps=data_length, verbose=1)
    predicted_speed.shape = dataset_frames_num-1
    
    a_meta = a_meta.assign(predicted_speed=pd.Series(np.empty((dataset_frames_num))).values)
    a_meta.loc[:, 'predicted_speed'] = np.nan
    a_meta.loc[1:,'predicted_speed'] = predicted_speed
    
    if setError:
        a_meta = a_meta.assign(error=pd.Series(np.empty((dataset_frames_num))).values)
        a_meta.loc[:]['error'] = np.nan
        a_meta.loc[1:,'error'] = abs(a_meta.loc[1:, 'speed'] - predicted_speed)

    a_meta.to_csv(a_meta_loc, index=False)

In [ ]:
predictSpeed(train_meta, model_train_meta_loc, False, True)

In [ ]:
train_meta = pd.read_csv(model_train_meta_loc)

In [ ]:
print(mean_squared_error(train_meta.loc[1:, 'speed'], train_meta.loc[1:, 'predicted_speed']))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(train_meta[:100])

In [ ]:
train_meta = train_meta.loc[:, :'error']

In [ ]:
train_meta.to_csv(model_train_meta_loc, index=False)

In [ ]:
np.mean(train_meta.loc[:,'error'])

In [ ]:
np.mean(train_meta.loc[:,'med_error'])

In [ ]:
np.mean(train_meta.loc[:,'mean_error'])

In [ ]:
print(train_meta.loc[20376:, 'predicted_speed'])

In [ ]:
weighted_range = 50 # was 50
#train_meta = train_meta.assign(med_prd_spd=pd.Series(np.empty((20400))).values)
train_meta.loc[:, 'med_prd_spd'] = np.nan
train_meta.loc[1:,'med_prd_spd'] = train_meta[1:]['predicted_speed'].rolling(weighted_range, center=True).median()
train_meta.loc[1:25, 'med_prd_spd'] = train_meta.loc[1:25, 'predicted_speed']
train_meta.loc[20376:, 'med_prd_spd'] = train_meta.loc[20376:, 'predicted_speed']
#train_meta.to_csv(model_train_meta_loc, index=False)

In [ ]:
weighted_range = 50 # was 50
#train_meta = train_meta.assign(mean_prd_spd=pd.Series(np.empty((20400))).values)
train_meta.loc[:, 'mean_prd_spd'] = np.nan
train_meta.loc[1:,'mean_prd_spd'] = train_meta[1:]['predicted_speed'].rolling(weighted_range, center=True).mean()
train_meta.loc[1:25, 'mean_prd_spd'] = train_meta.loc[1:25, 'predicted_speed']
train_meta.loc[20376:, 'mean_prd_spd'] = train_meta.loc[20376:, 'predicted_speed']
#train_meta.to_csv(model_train_meta_loc, index=False)

In [ ]:
#train_meta = train_meta.assign(med_error=pd.Series(np.empty((20400))).values)
train_meta.loc[:]['med_error'] = np.nan
train_meta.loc[1:,'med_error'] = abs(train_meta.loc[1:, 'speed'] - train_meta.loc[1:,'med_prd_spd'])

In [ ]:
#train_meta = train_meta.assign(mean_error=pd.Series(np.empty((20400))).values)
train_meta.loc[:]['mean_error'] = np.nan
train_meta.loc[1:,'mean_error'] = abs(train_meta.loc[1:, 'speed'] - train_meta.loc[1:,'mean_prd_spd'])

In [ ]:
train_meta.head()

In [ ]:
train_meta.to_csv(model_train_meta_loc, index=False)

In [ ]:
print(np.mean(train_meta.loc[:,'error']))
print(np.mean(train_meta.loc[:,'med_error']))
print(np.mean(train_meta.loc[:,'mean_error']))
print()
print(mean_squared_error(train_meta.loc[1:, 'speed'], train_meta.loc[1:, 'predicted_speed']))
print(mean_squared_error(train_meta.loc[1:, 'speed'], train_meta.loc[1:, 'med_prd_spd']))
print(mean_squared_error(train_meta.loc[1:, 'speed'], train_meta.loc[1:, 'mean_prd_spd']))

In [ ]:
data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
dataset_frames_num = train_meta.shape[0]
#train_frames_num = int(.8*(dataset_frames_num-1))
train_frames_num = int(.6*(dataset_frames_num-1))
val_frames_num = int(.5*((dataset_frames_num-1) - train_frames_num))
test_frames_num = int(((dataset_frames_num-1) - (train_frames_num + val_frames_num)))
print(train_frames_num, val_frames_num, test_frames_num, train_frames_num+val_frames_num+test_frames_num)

train_index = data_index[:train_frames_num]
val_index = data_index[train_frames_num:(train_frames_num+val_frames_num)]
test_index = data_index[(train_frames_num+val_frames_num):]

In [ ]:
test1 = 0
test2 = 0
test3 = 0
for i in train_index:
    test1 = test1 + train_meta.loc[i, 'error']
test1_loss = test1/train_frames_num
for i in val_index:
    test2 = test2 + train_meta.loc[i, 'error']
test2_loss = test2/val_frames_num
for i in test_index:
    test3 = test3 + train_meta.loc[i, 'error']
test3_loss = test3/test_frames_num

In [ ]:
print(test1_loss, test2_loss, test3_loss)

In [ ]:
test_meta = pd.read_csv(test_meta_loc)

In [ ]:
predictSpeed(test_meta, model_test_meta_loc, False, False)

In [ ]:
test_meta = pd.read_csv(model_test_meta_loc)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(test_meta[:100])

In [ ]:
weighted_range = 100 # was 50
#test_meta = test_meta.assign(med_prd_spd=pd.Series(np.empty((20400))).values)
test_meta.loc[:, 'med_prd_spd'] = np.nan
test_meta.loc[1:,'med_prd_spd'] = test_meta[1:]['predicted_speed'].rolling(weighted_range, center=True).median()
test_meta.loc[1:25, 'med_prd_spd'] = test_meta.loc[1:25, 'predicted_speed']
test_meta.loc[10774:, 'med_prd_spd'] = test_meta.loc[10774:, 'predicted_speed']
#test_meta.to_csv(model_test_meta_loc, index=False)

In [ ]:
weighted_range = 100 # was 50
#test_meta = test_meta.assign(mean_prd_spd=pd.Series(np.empty((20400))).values)
test_meta.loc[:, 'mean_prd_spd'] = np.nan
test_meta.loc[1:,'mean_prd_spd'] = test_meta[1:]['predicted_speed'].rolling(weighted_range, center=True).mean()
test_meta.loc[1:25, 'mean_prd_spd'] = test_meta.loc[1:25, 'predicted_speed']
test_meta.loc[10774:, 'mean_prd_spd'] = test_meta.loc[10774:, 'predicted_speed']
#test_meta.to_csv(model_test_meta_loc, index=False)

In [ ]:
test_meta.to_csv(model_test_meta_loc, index=False)

In [ ]:
stop_length = test_meta.loc[1080:1640].shape[0]

In [ ]:
test1 = 0
test2 = 0
test3 = 0
for i in test_meta.loc[1080:1640, 'predicted_speed']:
    test1 = test1 + i
test1_loss = test1/stop_length
for i in test_meta.loc[1080:1640, 'med_prd_spd']:
    test2 = test2 + i
test2_loss = test2/stop_length
for i in test_meta.loc[1080:1640, 'mean_prd_spd']:
    test3 = test3 + i
test3_loss = test3/stop_length

In [ ]:
print(test1_loss, test2_loss, test3_loss)

In [ ]:
test_result = test_meta.loc[:, 'med_prd_spd']

In [ ]:
type(test_result)

In [ ]:
test_result.to_csv(test_result_path, index=False)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
print(fig_size)
fig_size[0] = 15
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
fig_size2 = plt.rcParams["figure.figsize"]
print(fig_size2)

In [ ]:
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'predicted_speed'], 'bx')
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'speed'], 'ro')

In [ ]:
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'med_prd_spd'], 'bx')
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'speed'], 'r')

In [ ]:
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'mean_prd_spd'], 'bx')
plt.plot(train_meta.loc[1:, 'image_index'], train_meta.loc[1:, 'speed'], 'ro')

In [ ]:
plt.plot(test_meta.loc[1:, 'image_index'], test_meta.loc[1:, 'predicted_speed'], 'bx')

In [ ]:
plt.plot(test_meta.loc[1:, 'image_index'], test_meta.loc[1:, 'med_prd_spd'])

In [ ]:
plt.plot(test_meta.loc[1:, 'image_index'], test_meta.loc[1:, 'mean_prd_spd'], 'bx')